In [239]:
# Dependencies
from bs4 import BeautifulSoup
import requests



In [214]:
# URL of page to be scraped
#url = 'https://www.walgreens.com/store/c/halloween-candy/ID=520931-tier3'
url = 'https://www.walgreens.com/store/store/category/productlist.jsp?N=520931&Eon=520931'


In [215]:
# https://www.walgreens.com/store/store/category/productlist.jsp?N=520931&Eon=520931&No=144
# &No144
# there's 174 total 

In [216]:
# Retrieve page with the requests module
response = requests.get(url)

In [217]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [218]:
# Examine the results, then determine element that contains sought info
#soup

In [219]:
# results are returned as an iterable list
#results = soup.find_all('li', class_="result-row")



In [220]:
# <div class="brand" id="prodBrandNamecompare_sku6253945" data-reactid="468">PayDay</div>
# "brand"
#<input type="checkbox" id="filter_353669" tabindex="0" value="353669" name="353669" title="Sour Punch(2)">
brands = soup.find_all("div", class_="brand")
print(brands)

[<div class="brand" data-reactid="481" id="prodBrandNamecompare_sku6253945">PayDay</div>, <div class="brand" data-reactid="532" id="prodBrandNamecompare_sku6062317">Snickers</div>, <div class="brand" data-reactid="581" id="prodBrandNamecompare_sku6180836">Tootsie Roll</div>, <div class="brand" data-reactid="634" id="prodBrandNamecompare_sku6275845">Milky Way</div>, <div class="brand" data-reactid="688" id="prodBrandNamecompare_sku4266421">Reese's</div>, <div class="brand" data-reactid="739" id="prodBrandNamecompare_sku1594278">Mounds</div>, <div class="brand" data-reactid="790" id="prodBrandNamecompare_sku6239204">Hershey's</div>, <div class="brand" data-reactid="845" id="prodBrandNamecompare_sku6329541">Kisses</div>]


In [221]:
for brand in brands:
    print(brand.text)

PayDay
Snickers
Tootsie Roll
Milky Way
Reese's
Mounds
Hershey's
Kisses


In [222]:
#<a name="filter_by_Brand" class="no-toggle title__master dt-selected" id="filter_Brand" href="javascript:void(0)" aria-expanded="true" data-reactid="173"><strong data-reactid="174"><legend data-reactid="175">Brand</legend></strong></a>
# type something that allows you select the a element (link that has the id)
# for the a element, find the next sibling
# tier list of things, nested inside things, a tree
# a , div, div, ul, li, 

In [223]:
labels = soup.find_all('li', 'check-box')
print(labels)

for label in labels:
    print(label.text)

[<li class="check-box" data-reactid="112"><input data-reactid="113" id="filter_2000011813" name="2000011813" tabindex="0" title="Candy(78)" type="checkbox" value="2000011813"/><label data-reactid="114" for="filter_2000011813"><span class="check" data-reactid="115"></span><span class="text-truncate" data-reactid="116">Candy</span><!-- react-text: 117 --> (<!-- /react-text --><!-- react-text: 118 -->78<!-- /react-text --><!-- react-text: 119 -->)<!-- /react-text --></label></li>, <li class="check-box" data-reactid="120"><input data-reactid="121" id="filter_2000013458" name="2000013458" tabindex="0" title="Chocolate - Bag(49)" type="checkbox" value="2000013458"/><label data-reactid="122" for="filter_2000013458"><span class="check" data-reactid="123"></span><span class="text-truncate" data-reactid="124">Chocolate - Bag</span><!-- react-text: 125 --> (<!-- /react-text --><!-- react-text: 126 -->49<!-- /react-text --><!-- react-text: 127 -->)<!-- /react-text --></label></li>, <li class="chec

In [229]:
what = soup.label.parent
print(what.text)

Search by keyword or item #Closing Suggestion 


In [230]:
filterA = soup.find('a',id='filter_Brand')
print(filterA)

<a aria-expanded="true" class="no-toggle title__master dt-selected selected" data-reactid="173" href="javascript:void(0)" id="filter_Brand" name="filter_by_Brand"><strong data-reactid="174"><legend data-reactid="175">Brand</legend></strong></a>


In [237]:
parentDiv = filterA.find_next_sibling('div')
labeleds = parentDiv.find_all(class_="text-truncate")
for labeled in labeleds:
    print(labeled.text)

Hershey's
Mars
Reese's
Brach's
Russell Stover
Kit Kat
M&M's


In [ ]:
#splinter is what's used to navigate a page
